In [1]:
import requests
from bs4 import BeautifulSoup
from scrapingbee import ScrapingBeeClient
import math
import pandas as pd
from datetime import datetime, timedelta


In [2]:
def scrape_page(url):
    client = ScrapingBeeClient(api_key='7RGXHBFZ5Z598EJFDXJT5F7JMXY15EGTQ2FSDKX53QL8Y8G8975BPAY1LT0EH86WRFI3WOOK75ILGQYE')
    response = client.get(url)
    soup = BeautifulSoup(response.content,'html.parser')
    return soup

In [4]:
def scrape_one_page(job_links):
    data_list = []
    
    for job_list in job_links:
        single_page_scrape = scrape_page(job_list)
        #Job_Title
        job_title = single_page_scrape.h2
        if job_title is not None:
            job_title = job_title.get_text('h2').strip()
        else:
            job_title = " "

        #Job Url
        job_url = job_list

        #location
        location = single_page_scrape.find('h6', {'class': 'location inline-block m-r-30 m-t-0'})
        if location is not None:
            location_txt = location.text.strip()
        else:
            location_txt = " "
        
        # posted

        job_posted_time = None  # Initialize the variable with a default value
        posted_element = single_page_scrape.find('h6', class_='semi-bold date-posted')

        if posted_element is not None:
            job_posted_time = posted_element.text.strip()
        else:
            job_posted_time = ""

        # Define the date format
        date_format = "%Y-%m-%d"

        # Check if the time is given in days
        if "Posted days" in job_posted_time:
            # Subtract the corresponding number of days from the current date
            job_posted_date = datetime.now() - timedelta(days=int(job_posted_time.split()[0]))
            # Format the resulting date as a string
            job_posted_dates = job_posted_date.strftime(date_format)
        # Check if the time is given in hours
        elif "Posted hours" in job_posted_time:
            # Subtract the corresponding number of hours from the current date
            job_posted_date = datetime.now() - timedelta(hours=int(job_posted_time.split()[0]))
            # Format the resulting date as a string
            job_posted_dates = job_posted_date.strftime(date_format)
        # If the time is not given in days or hours, use the current date
        else:
            job_posted_dates = datetime.now()

        # Print the formatted date
        print(job_posted_dates)
        
        # description

        job_description = single_page_scrape.find("div", class_="main-content")
        if job_description is not None:
            description = job_description.find("div", class_="job-description hidden-text shadow")
            if description is not None:
                description_txt = description.text.strip()
            else:
                description_txt = ""
        else:
            description_txt = ""

        data = {
            'job_title':job_title,
            'job_url':job_url,
            'location':location_txt,
            'posted':job_posted_dates,
            'description':description_txt

           
        }
        data_list.append(data)
    print(data_list)
    return data_list


In [18]:
job_to_scrape = 'Data Engineer'
l='Texas'
distance = 10
posted = "1d"
scrape = scrape_page(f"https://search.linkup.com/search/results/data-engineer-jobs-in-texas?pageNum=1&posted=7d")


roles = ["Data engineer", "Machine learning"]
cleaned_role = [role.replace(" ", "-") for role in roles]

In [15]:
#To find page number
search_details = scrape.find('div', {'class': 'search-details'})
job_count_text = search_details.find('div').text.strip()
job_count = int(job_count_text.split()[-2].replace(',', ''))
page_number = math.ceil((job_count)/25)
print(job_count)
page_number


1190


48

In [20]:
job_links = []
i = 1

for cleaned_roles in cleaned_role:


    while i <= 3:
        print('Page number', i)
        scrape_result = scrape_page(f"https://search.linkup.com/search/results/{cleaned_roles}-jobs-in-texas?pageNum={i}&posted=7d")

        for link in scrape_result.find_all('a', class_='organic-link search-result-link'):
            x = "https://search.linkup.com"+link.get('href')
            job_links.append(x)

        # for j in range(len(job_links)):
        #     job_links[j] = 'https://search.linkup.com' + job_links[j]

        i = i + 1
        print(job_links)


Page number 1
['https://search.linkup.com/details/6eeaf54901814c7b69ebcec610324802', 'https://search.linkup.com/details/969f19899c2311c01b61461fcd040bce', 'https://search.linkup.com/details/052ed23cc8354f9b78d7df577ecb1717', 'https://search.linkup.com/details/fd1b92f4facb5bfabcb2d01311e4442e', 'https://search.linkup.com/details/e6bdfe673e3e719b82f7e20fbb6ef14f', 'https://search.linkup.com/details/51e8c63cda388b6293599c139e814975', 'https://search.linkup.com/details/645e51650bdc3b8eea13073772892acf', 'https://search.linkup.com/details/0886b61a8431957ac34a5d462eaa1a8f', 'https://search.linkup.com/details/5b8578a2679225298b96f7a61599b29d', 'https://search.linkup.com/details/226279dd270a59d9fec3928b6cf0379b', 'https://search.linkup.com/details/6cc8acb14cfa933e6ddb1a55ce78ff1a', 'https://search.linkup.com/details/8cab58147bafacf0b3a3f29dc5a5cbcc', 'https://search.linkup.com/details/97c6da0f6fa1eff39d20351f74498d8c', 'https://search.linkup.com/details/154024681efab3b43b738f9b2d34458d', 'http

In [22]:
page_data = scrape_one_page(job_links)

2023-07-24 15:51:12.714567
2023-07-24 15:51:17.681848
2023-07-24 15:51:22.133385
2023-07-24 15:51:42.792081
2023-07-24 15:51:47.068751
2023-07-24 15:51:51.988506
2023-07-24 15:51:56.698928
2023-07-24 15:52:01.506645
2023-07-24 15:52:07.351960
2023-07-24 15:52:12.332690
2023-07-24 15:52:17.075797
2023-07-24 15:52:21.269117
2023-07-24 15:52:26.113647
2023-07-24 15:52:30.279432
2023-07-24 15:52:34.994176
2023-07-24 15:52:40.415299
2023-07-24 15:52:44.621391
2023-07-24 15:52:49.254565
2023-07-24 15:52:53.407355
2023-07-24 15:52:57.893722
2023-07-24 15:53:02.427402
2023-07-24 15:53:06.939821
2023-07-24 15:53:11.962738
2023-07-24 15:53:16.419582
2023-07-24 15:53:22.295821
2023-07-24 15:53:26.296478
2023-07-24 15:53:31.315174
2023-07-24 15:53:39.739559
2023-07-24 15:53:43.906100
2023-07-24 15:53:48.721781
2023-07-24 15:53:53.530939
2023-07-24 15:53:58.249638
2023-07-24 15:54:02.430012
2023-07-24 15:54:07.044600
2023-07-24 15:54:11.348942
2023-07-24 15:54:16.056017
2023-07-24 15:54:20.095044
2

In [21]:
#Converting to dataframe
df = pd.DataFrame(page_data)
df.to_csv('linkup.csv',index=False)
df

,job_title,job_url,location,posted,description
0,Data Engineer,https://search.linkup.com/details/6eeaf5490181...,"Plano, TX",2023-07-24 15:33:57.656435,Req ID: 246917 \nNTT DATA Services strives to ...
1,Data Engineer,https://search.linkup.com/details/969f19899c23...,"Houston, TX",2023-07-24 15:34:01.933090,Job Title: (Back End) Data Engineer\nLocation:...
2,Data Engineer,https://search.linkup.com/details/052ed23cc835...,"Irving, TX",2023-07-24 15:34:06.239552,"At Greystar, we've launched a program aimed at..."
3,Data Engineer,https://search.linkup.com/details/fd1b92f4facb...,"Austin, TX",2023-07-24 15:34:10.839136,"Position Summary\nAre you an experienced, pass..."
4,Data Engineer II,https://search.linkup.com/details/e6bdfe673e3e...,"Austin, TX",2023-07-24 15:34:15.858708,Company: \nCox Automotive - USA\nJob Family Gr...
...,...,...,...,...,...
70,Quality Engineer,https://search.linkup.com/details/0ec614fb5b74...,"Fort Worth, TX",2023-07-24 15:39:54.293827,"We are global, we are impacting the lives of m..."
71,Materials Engineer,https://search.linkup.com/details/31881776170c...,"Katy, TX",2023-07-24 15:39:58.488624,The Materials Engineer is responsible for eval...
72,Network Engineer,https://search.linkup.com/details/ab06b15b3391...,TX,2023-07-24 15:40:02.635328,NTT is a leading global IT solutions and servi...
73,Software Engineer,https://search.linkup.com/details/555fc6d02699...,TX,2023-07-24 15:40:07.398689,SAIC is seeking qualified Software Engineers f...
